In [1]:
import os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import random

In [2]:
random.seed(42)

In [3]:
dataset_path = 'dataset-final/main.csv'
dataset_images = 'dataset-final/skin-disease-dataset'

In [4]:
df=pd.read_csv(dataset_path)
print(df.head())

    Image Path  KA  KB  CA  DT  M  L  MN
0  KA_0001.jpg   1   0   0   0  0  0   0
1  KA_0002.jpg   1   0   0   0  0  0   0
2  KA_0003.jpg   1   0   0   0  0  0   0
3  KA_0004.jpg   1   0   0   0  0  0   0
4  KA_0005.jpg   1   0   0   0  0  0   0


In [5]:
image_size = (128, 128)
X = []
y = []

# Loop through the dataset to load images and their corresponding labels
for index, row in df.iterrows():
    img_path = os.path.join(dataset_images, row['Image Path'])
    img = cv2.imread(img_path)
    img = cv2.resize(img, image_size)  # Resize image to the defined size
    X.append(img)
    y.append([row['KA'], row['KB'], row['CA'], row['DT'], row['M'], row['L'], row['MN']])

# Convert X and y to numpy arrays
X = np.array(X)
y = np.array(y)

# Normalize the image data
X = X.astype('float32') / 255.0

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Initialize the CNN model
model = Sequential()

# Add convolutional and pooling layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

# Add fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(7, activation='softmax'))  # 7 classes for the multi-label classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()


C:\Users\Alex\anaconda3\envs\ml2\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       7,372,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,393,223 (28.20 MB)

 Trainable params: 7,393,223 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Split data manually (instead of using validation_split)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train the model with explicit validation data
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


# Save the trained model
model.save('skin_disease_model.h5')


Epoch 1/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 35s 181ms/step - accuracy: 0.1690 - loss: 2.0052 - val_accuracy: 0.1789 - val_loss: 1.9201
Epoch 2/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 31s 173ms/step - accuracy: 0.1955 - loss: 1.9147 - val_accuracy: 0.1824 - val_loss: 1.9130
Epoch 3/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 31s 173ms/step - accuracy: 0.2095 - loss: 1.8962 - val_accuracy: 0.2060 - val_loss: 1.8997
Epoch 4/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 32s 177ms/step - accuracy: 0.2243 - loss: 1.8807 - val_accuracy: 0.2108 - val_loss: 1.9016
Epoch 5/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 31s 170ms/step - accuracy: 0.2267 - loss: 1.8647 - val_accuracy: 0.2108 - val_loss: 1.8851
Epoch 6/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 31s 173ms/step - accuracy: 0.2457 - loss: 1.8424 - val_accuracy: 0.2129 - val_loss: 1.8956
Epoch 7/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 31s 173ms/step - accuracy: 0.2585 - loss: 1.8116 - val_accuracy: 0.2247 - val_loss: 1.8877
Epoch 8/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 31s 173ms/step - accuracy: 0.2846 - loss: 1

In [9]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.2484 - loss: 1.9178
Test Accuracy: 23.57%
